<a href="https://colab.research.google.com/github/KrzRac/UGP/blob/main/enkoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')

drive_results_dir = "/content/drive/MyDrive/roberta_results"
os.makedirs(drive_results_dir, exist_ok=True)

Mounted at /content/drive


In [4]:
dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenized_datasets = dataset.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [7]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})


In [8]:
train_dataset = tokenized_datasets["train"]
valid_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

In [9]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
frozen_layer_count = 4  # Number of encoder layers to freeze
for name, param in model.named_parameters():
    if "embeddings" in name or any(f"layer.{i}." in name for i in range(frozen_layer_count)):
        param.requires_grad = False

In [11]:
print("RobertaForSequenceClassification:")
print(model)
frozen_layers = [name for name, param in model.named_parameters() if not param.requires_grad]
print("Frozen layers:", frozen_layers)

RobertaForSequenceClassification:
RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

In [13]:
training_args = TrainingArguments(
    output_dir=drive_results_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir=drive_results_dir,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-af6bbc48ea2a>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
print("Training RobertaForSequenceClassification...")
trainer.train()

Training RobertaForSequenceClassification...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.356100,0.319464,0.874296
2,0.202700,0.329604,0.884615
3,0.202500,0.311504,0.899625
4,0.121700,0.388592,0.902439
5,0.205300,0.466741,0.890244


TrainOutput(global_step=2670, training_loss=0.23983121538876595, metrics={'train_runtime': 607.2974, 'train_samples_per_second': 70.229, 'train_steps_per_second': 4.397, 'total_flos': 1718852108011320.0, 'train_loss': 0.23983121538876595, 'epoch': 5.0})

In [15]:
eval_results = trainer.evaluate(test_dataset)

eval_results_file = os.path.join(drive_results_dir, "eval_results.txt")
with open(eval_results_file, "w") as f:
    f.write(str(eval_results))

print(f"Evaluation results saved at: {eval_results_file}")

print("Evaluation results:", eval_results)

Evaluation results saved at: /content/drive/MyDrive/roberta_results/eval_results.txt
Evaluation results: {'eval_loss': 0.48724570870399475, 'eval_accuracy': 0.8855534709193246, 'eval_runtime': 4.0811, 'eval_samples_per_second': 261.207, 'eval_steps_per_second': 16.417, 'epoch': 5.0}


In [16]:
print("Generating detailed evaluation report...")
logits, labels = trainer.predict(test_dataset)[:2]
predictions = np.argmax(logits, axis=-1)
report = classification_report(labels, predictions, target_names=["negative", "positive"], digits=4)

classification_report_file = os.path.join(drive_results_dir, "classification_report.txt")
with open(classification_report_file, "w") as f:
    f.write(report)

print(f"Classification report saved at: {classification_report_file}")

print("Classification Report:\n", report)

Generating detailed evaluation report...


Classification report saved at: /content/drive/MyDrive/roberta_results/classification_report.txt
Classification Report:
               precision    recall  f1-score   support

    negative     0.8757    0.8987    0.8870       533
    positive     0.8960    0.8724    0.8840       533

    accuracy                         0.8856      1066
   macro avg     0.8858    0.8856    0.8855      1066
weighted avg     0.8858    0.8856    0.8855      1066



In [17]:
model_save_path = os.path.join(drive_results_dir, "roberta_finetuned")
tokenizer_save_path = os.path.join(drive_results_dir, "roberta_finetuned_tokenizer")

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved at: {model_save_path}")
print(f"Tokenizer saved at: {tokenizer_save_path}")

Model saved at: /content/drive/MyDrive/roberta_results/roberta_finetuned
Tokenizer saved at: /content/drive/MyDrive/roberta_results/roberta_finetuned_tokenizer


In [18]:
examples = test_dataset['text'][:3]

tokens = tokenizer(examples, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Ensure the model and data are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokens = {key: val.to(device) for key, val in tokens.items()}

# Get predictions
outputs = model(**tokens)
predictions = torch.argmax(outputs.logits, dim=-1)

print("Predictions:")
for i, example in enumerate(examples):
    print(f"Review: {example}")
    print(f"Predicted sentiment: {'positive' if predictions[i] == 1 else 'negative'}")


Predictions:
Review: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
Predicted sentiment: positive
Review: consistently clever and suspenseful .
Predicted sentiment: positive
Review: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .
Predicted sentiment: negative
